# Homework #4: Pop Music Transformer Colab notebook

Generate pop piano music using Transformer models. In this task, we use REMI, which stands for `REvamped MIDI-derived events.` 

REMI is a new event representation for converting MIDI scores into text-like discrete tokens. 

- Experience MIDI, REMI data processing
- Experience Transformer architecture


# Ready to start!


Google Drive Mount


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Create a new working folder

Move to your working directory with '%cd' command.

In [2]:
%cd /content/drive/MyDrive/GCT634_AI613

/content/drive/MyDrive/GCT634_AI613


Clone git repository
https://github.com/juhannam/gct634-ai613-2021/HW4

In [3]:
%cd HW4/

/content/drive/MyDrive/GCT634_AI613/HW4


Install Dependencies
Check the README for required dependencies and install them.

** Colab already has tensorflow. However, since version 1.14 is required, re-installation may be required.

In [4]:
!pip install miditoolkit
!pip uninstall tensorflow
!pip install tensorflow-gpu==1.14

     |████████████████████████████████| 51 kB 3.9 MB/s 
Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.7.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.7.0
     |████████████████████████████████| 377.1 MB 9.4 kB/s 
     |████████████████████████████████| 488 kB 51.3 MB/s 
     |████████████████████████████████| 50 kB 6.9 MB/s 
     |████████████████████████████████| 3.1 MB 49.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:


In [5]:
import utils
import modules
import model
import chord_recognition
import miditoolkit


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 



---



# Practice 1 : MIDI to Event representation conversion


## Read MIDI


In [ ]:
file_name = 'sample_data/[Claude Debussy][Suite Bergamasque].midi' # 'sample_data/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R1_2014_wav--1.midi'
midi_obj = miditoolkit.midi.parser.MidiFile(file_name)

In [ ]:

print('-MIDI META DATA-')
print(midi_obj)
print()
print('-Instrument information(GM number)-')
print(*midi_obj.instruments, sep='\n')

-MIDI META DATA-
ticks per beat: 480
max tick: 905196
tempo changes: 1
time sig: 0
key sig: 0
markers: 0
lyrics: False
instruments: 1

-Instrument information(GM number)-
Instrument(program=0, is_drum=False, name="")


Print note events
- show 10 note events of instrument 0
- https://www.inspiredacoustics.com/en/MIDI_note_numbers_and_center_frequencies

In [ ]:
print(*midi_obj.instruments[0].notes[:10], sep='\n')

Note(start=1628, end=2027, pitch=48, velocity=127)
Note(start=2672, end=2703, pitch=70, velocity=127)
Note(start=2672, end=3747, pitch=74, velocity=127)
Note(start=890, end=4331, pitch=29, velocity=127)
Note(start=1259, end=4454, pitch=41, velocity=127)
Note(start=4853, end=4915, pitch=82, velocity=127)
Note(start=4608, end=5007, pitch=81, velocity=127)
Note(start=5007, end=5222, pitch=81, velocity=127)
Note(start=5099, end=5314, pitch=79, velocity=127)
Note(start=2027, end=5498, pitch=48, velocity=127)


## Convert to REMI events

REMI, which stands for REvamped MIDI-derived events, is a new event representation we propose for converting MIDI scores into text-like discrete tokens. Compared to the MIDI-like event representation adopted in exising Transformer-based music composition models, REMI provides sequence models a metrical context for modeling the rhythmic patterns of music. 

Read midi into "Item"
- Item is container class of events defined in utils.py

In [ ]:
note_items, tempo_items = utils.read_items(file_name)

In [ ]:
print(*note_items[:10], sep='\n')

Item(name=Note, start=890, end=4331, velocity=127, pitch=29)
Item(name=Note, start=1259, end=4454, velocity=127, pitch=41)
Item(name=Note, start=1628, end=2027, velocity=127, pitch=48)
Item(name=Note, start=2027, end=5498, velocity=127, pitch=48)
Item(name=Note, start=2641, end=5775, velocity=127, pitch=58)
Item(name=Note, start=2641, end=5928, velocity=127, pitch=62)
Item(name=Note, start=2672, end=2703, velocity=127, pitch=70)
Item(name=Note, start=2672, end=3747, velocity=127, pitch=74)
Item(name=Note, start=4608, end=5007, velocity=127, pitch=81)
Item(name=Note, start=4853, end=4915, velocity=127, pitch=82)


In [ ]:
print(*tempo_items[:10], sep='\n')

Item(name=Tempo, start=0, end=None, velocity=None, pitch=120)


Quantize note items

In [ ]:
note_items = utils.quantize_items(note_items, ticks=64) 

In [ ]:
print(*note_items[:10], sep='\n')

Item(name=Note, start=896, end=4337, velocity=127, pitch=29)
Item(name=Note, start=1280, end=4475, velocity=127, pitch=41)
Item(name=Note, start=1600, end=1999, velocity=127, pitch=48)
Item(name=Note, start=2048, end=5519, velocity=127, pitch=48)
Item(name=Note, start=2624, end=5758, velocity=127, pitch=58)
Item(name=Note, start=2624, end=5911, velocity=127, pitch=62)
Item(name=Note, start=2688, end=2719, velocity=127, pitch=70)
Item(name=Note, start=2688, end=3763, velocity=127, pitch=74)
Item(name=Note, start=4608, end=5007, velocity=127, pitch=81)
Item(name=Note, start=4864, end=4926, velocity=127, pitch=82)


extract chord (optional)
- we can use rule-based chord extraction using chord_recognition.py
- chord recognition resolution : 2 beat ~ 4 beat

In [ ]:
chord_items = utils.extract_chords(note_items) 

In [ ]:
print(*chord_items[:10], sep='\n')

Item(name=Chord, start=0, end=3840, velocity=None, pitch=A#:maj)
Item(name=Chord, start=3840, end=4800, velocity=None, pitch=A#:maj)
Item(name=Chord, start=4800, end=5760, velocity=None, pitch=A#:maj)
Item(name=Chord, start=5760, end=7680, velocity=None, pitch=F:maj)
Item(name=Chord, start=7680, end=8640, velocity=None, pitch=D:min)
Item(name=Chord, start=8640, end=9600, velocity=None, pitch=A:min)
Item(name=Chord, start=9600, end=11520, velocity=None, pitch=A#:maj)
Item(name=Chord, start=11520, end=13440, velocity=None, pitch=D:min)
Item(name=Chord, start=13440, end=15360, velocity=None, pitch=F:maj)
Item(name=Chord, start=15360, end=17280, velocity=None, pitch=G:min)


group items

In [ ]:
items = chord_items + tempo_items + note_items
max_time = note_items[-1].end
groups = utils.group_items(items, max_time)

In [ ]:
for g in groups:
    print(*g, sep='\n')
    print()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Item(name=Chord, start=365760, end=366720, velocity=None, pitch=G:maj)
Item(name=Note, start=365760, end=366067, velocity=127, pitch=57)
Item(name=Note, start=365824, end=365855, velocity=127, pitch=78)
Item(name=Note, start=365824, end=365855, velocity=127, pitch=84)
Item(name=Note, start=365824, end=366408, velocity=127, pitch=72)
Item(name=Note, start=366464, end=366495, velocity=127, pitch=55)
Item(name=Note, start=366528, end=368709, velocity=127, pitch=71)
Item(name=Note, start=366528, end=368678, velocity=127, pitch=67)
366720

366720
Item(name=Chord, start=366720, end=368640, velocity=None, pitch=E:min)
Item(name=Note, start=366912, end=367343, velocity=127, pitch=74)
Item(name=Note, start=367232, end=367263, velocity=127, pitch=45)
Item(name=Note, start=367232, end=367263, velocity=127, pitch=52)
Item(name=Note, start=367296, end=367726, velocity=127, pitch=78)
Item(name=Note, start=368000, end=368031, velocity=127, pitch=76)
Item(name=Note,

In [ ]:
events = utils.item2event(groups)

In [ ]:
print(*events[:30], sep='\n')

Event(name=Bar, time=None, value=None, text=1)
Event(name=Position, time=0, value=1/16, text=0)
Event(name=Chord, time=0, value=A#:maj, text=A#:maj)
Event(name=Position, time=0, value=1/16, text=0)
Event(name=Tempo Class, time=0, value=mid, text=None)
Event(name=Tempo Value, time=0, value=30, text=None)
Event(name=Position, time=896, value=8/16, text=896)
Event(name=Note Velocity, time=896, value=31, text=127/124)
Event(name=Note On, time=896, value=29, text=29)
Event(name=Note Duration, time=896, value=56, text=3441/3420)
Event(name=Position, time=1280, value=12/16, text=1280)
Event(name=Note Velocity, time=1280, value=31, text=127/124)
Event(name=Note On, time=1280, value=41, text=41)
Event(name=Note Duration, time=1280, value=52, text=3195/3180)
Event(name=Position, time=1600, value=14/16, text=1600)
Event(name=Note Velocity, time=1600, value=31, text=127/124)
Event(name=Note On, time=1600, value=48, text=48)
Event(name=Note Duration, time=1600, value=6, text=399/420)
Event(name=Bar

Save Events as a file

In [ ]:
from collections import Counter
import pickle

pickle.dump(events, open('events.pkl', 'wb'))

Load saved events file



In [ ]:
events = pickle.load(open('events.pkl', 'rb'))

Convert events to MIDI

In [ ]:
midi_path = 'sample_data/[Claude Debussy][Suite Bergamasque]_restored.midi'

utils.write_midi_events(None, None, midi_path, prompt_path=None, events=events)

sample_data/[Claude Debussy][Suite Bergamasque]_restored.midi




---

\

# Task 1  : MIDI to Event representation conversion

1. Find your favorite piano songs
2. Transcribe the piano songs using the pre-trained onset and frames model or your own model (HW3)
3. Import your MIDI files, convert it to REMI and convert it back to MIDI file

You can try adjusting the variables. e.g. Modify the quantize_items' parameter

You should submit 

- Transcribed MIDI file
- REMI events.pkl
- Restored MIDI file(quantized)


In [16]:
import random 

def Manually_Change_Velocity(file_name):
  print('finetune_train_data/Original/'+file_name+'.midi')
  note_items, tempo_items = utils.read_items('finetune_train_data/Original/'+file_name+'.midi') 
  print(*note_items[:10], sep='\n')
  
  for note_item in note_items:
    note_item.velocity=random.randint(20,40)
  print('------------------------------------------------------------------------')
  print(*note_items[:10], sep='\n')
  print('\n')
  
  items = tempo_items + note_items
  max_time = note_items[-1].end
  groups = utils.group_items(items, max_time)

  events = utils.item2event(groups) 
  midi_path = 'finetune_train_data/Liszt/'+file_name+'_Velocity Changed'+'.midi'
  utils.write_midi_events(None, None, midi_path, prompt_path=None, events=events)

In [ ]:
file_name = 'sample_data/[Claude Debussy][Suite Bergamasque]'
Manually_Change_Velocity(file_name)

Item(name=Note, start=890, end=4331, velocity=127, pitch=29)
Item(name=Note, start=1259, end=4454, velocity=127, pitch=41)
Item(name=Note, start=1628, end=2027, velocity=127, pitch=48)
Item(name=Note, start=2027, end=5498, velocity=127, pitch=48)
Item(name=Note, start=2641, end=5775, velocity=127, pitch=58)
Item(name=Note, start=2641, end=5928, velocity=127, pitch=62)
Item(name=Note, start=2672, end=2703, velocity=127, pitch=70)
Item(name=Note, start=2672, end=3747, velocity=127, pitch=74)
Item(name=Note, start=4608, end=5007, velocity=127, pitch=81)
Item(name=Note, start=4853, end=4915, velocity=127, pitch=82)
------------------------------------------------------------------------
Item(name=Note, start=890, end=4331, velocity=30, pitch=29)
Item(name=Note, start=1259, end=4454, velocity=23, pitch=41)
Item(name=Note, start=1628, end=2027, velocity=33, pitch=48)
Item(name=Note, start=2027, end=5498, velocity=22, pitch=48)
Item(name=Note, start=2641, end=5775, velocity=33, pitch=58)
Item

In [17]:
import os 
file_list = os.listdir('finetune_train_data/Original/')
for file in file_list:
  if file.endswith(".midi"): 
    Manually_Change_Velocity(file.split(".midi")[0])

finetune_train_data/Original/[Franz Liszt][Concert Etude No. 2, Gnomenreigen, S. 1452].midi
Item(name=Note, start=921, end=952, velocity=127, pitch=62)
Item(name=Note, start=952, end=983, velocity=127, pitch=61)
Item(name=Note, start=1136, end=1167, velocity=127, pitch=66)
Item(name=Note, start=1167, end=1228, velocity=127, pitch=65)
Item(name=Note, start=1290, end=1351, velocity=127, pitch=69)
Item(name=Note, start=1320, end=1536, velocity=127, pitch=68)
Item(name=Note, start=1443, end=1505, velocity=127, pitch=69)
Item(name=Note, start=1566, end=1597, velocity=127, pitch=71)
Item(name=Note, start=1566, end=1597, velocity=127, pitch=73)
Item(name=Note, start=1689, end=1720, velocity=127, pitch=74)
------------------------------------------------------------------------
Item(name=Note, start=921, end=952, velocity=26, pitch=62)
Item(name=Note, start=952, end=983, velocity=28, pitch=61)
Item(name=Note, start=1136, end=1167, velocity=34, pitch=66)
Item(name=Note, start=1167, end=1228, ve

-------------------------------------------

In [ ]:
# Manually change
file_name = 'sample_data/[Domenico Scarlatti][Sonata in D Major, K. 118]' #'finetune_train_data/MIDI-Unprocessed_04_R3_2008_01-07_ORIG_MID--AUDIO_04_R3_2008_wav--7'
note_items, tempo_items = utils.read_items(file_name+'.midi')

In [ ]:
print(*note_items[:10], sep='\n')

Item(name=Note, start=952, end=1044, velocity=127, pitch=74)
Item(name=Note, start=1474, end=1597, velocity=127, pitch=81)
Item(name=Note, start=1935, end=2027, velocity=127, pitch=57)
Item(name=Note, start=2426, end=2918, velocity=127, pitch=62)
Item(name=Note, start=2887, end=3348, velocity=127, pitch=74)
Item(name=Note, start=2887, end=2949, velocity=127, pitch=78)
Item(name=Note, start=3348, end=3440, velocity=127, pitch=73)
Item(name=Note, start=3379, end=3471, velocity=127, pitch=76)
Item(name=Note, start=3502, end=3532, velocity=127, pitch=71)
Item(name=Note, start=3502, end=3563, velocity=127, pitch=74)


In [ ]:
import random
for note_item in note_items:
  note_item.velocity=random.randint(20,40)
  print(note_item)

Item(name=Note, start=952, end=1044, velocity=54, pitch=74)
Item(name=Note, start=1474, end=1597, velocity=37, pitch=81)
Item(name=Note, start=1935, end=2027, velocity=30, pitch=57)
Item(name=Note, start=2426, end=2918, velocity=55, pitch=62)
Item(name=Note, start=2887, end=3348, velocity=29, pitch=74)
Item(name=Note, start=2887, end=2949, velocity=60, pitch=78)
Item(name=Note, start=3348, end=3440, velocity=52, pitch=73)
Item(name=Note, start=3379, end=3471, velocity=60, pitch=76)
Item(name=Note, start=3502, end=3532, velocity=32, pitch=71)
Item(name=Note, start=3502, end=3563, velocity=51, pitch=74)
Item(name=Note, start=3624, end=3655, velocity=47, pitch=69)
Item(name=Note, start=3624, end=3686, velocity=50, pitch=73)
Item(name=Note, start=3747, end=3840, velocity=26, pitch=67)
Item(name=Note, start=3747, end=3809, velocity=56, pitch=71)
Item(name=Note, start=3870, end=3901, velocity=32, pitch=69)
Item(name=Note, start=3901, end=3932, velocity=38, pitch=66)
Item(name=Note, start=402

In [ ]:
print(*note_items[:10], sep='\n')

Item(name=Note, start=952, end=1044, velocity=54, pitch=74)
Item(name=Note, start=1474, end=1597, velocity=37, pitch=81)
Item(name=Note, start=1935, end=2027, velocity=30, pitch=57)
Item(name=Note, start=2426, end=2918, velocity=55, pitch=62)
Item(name=Note, start=2887, end=3348, velocity=29, pitch=74)
Item(name=Note, start=2887, end=2949, velocity=60, pitch=78)
Item(name=Note, start=3348, end=3440, velocity=52, pitch=73)
Item(name=Note, start=3379, end=3471, velocity=60, pitch=76)
Item(name=Note, start=3502, end=3532, velocity=32, pitch=71)
Item(name=Note, start=3502, end=3563, velocity=51, pitch=74)


In [ ]:
chord_items = utils.extract_chords(note_items)
print(*chord_items[:10], sep='\n')

Item(name=Chord, start=0, end=3840, velocity=None, pitch=D:maj)
Item(name=Chord, start=3840, end=4800, velocity=None, pitch=D:maj)
Item(name=Chord, start=4800, end=6720, velocity=None, pitch=B:min)
Item(name=Chord, start=6720, end=8640, velocity=None, pitch=D:maj)
Item(name=Chord, start=8640, end=10560, velocity=None, pitch=B:min)
Item(name=Chord, start=10560, end=12480, velocity=None, pitch=B:min)
Item(name=Chord, start=12480, end=14400, velocity=None, pitch=D:maj)
Item(name=Chord, start=14400, end=16320, velocity=None, pitch=B:min)
Item(name=Chord, start=16320, end=18240, velocity=None, pitch=B:min)
Item(name=Chord, start=18240, end=19200, velocity=None, pitch=D:maj)


In [ ]:
#items = chord_items + tempo_items + note_items
items = tempo_items + note_items
max_time = note_items[-1].end
groups = utils.group_items(items, max_time)
for g in groups:
    print(*g, sep='\n')
    print()

0
Item(name=Tempo, start=0, end=None, velocity=None, pitch=120)
Item(name=Note, start=952, end=1044, velocity=54, pitch=74)
Item(name=Note, start=1474, end=1597, velocity=37, pitch=81)
1920

1920
Item(name=Note, start=1935, end=2027, velocity=30, pitch=57)
Item(name=Note, start=2426, end=2918, velocity=55, pitch=62)
Item(name=Note, start=2887, end=3348, velocity=29, pitch=74)
Item(name=Note, start=2887, end=2949, velocity=60, pitch=78)
Item(name=Note, start=3348, end=3440, velocity=52, pitch=73)
Item(name=Note, start=3379, end=3471, velocity=60, pitch=76)
Item(name=Note, start=3502, end=3532, velocity=32, pitch=71)
Item(name=Note, start=3502, end=3563, velocity=51, pitch=74)
Item(name=Note, start=3624, end=3655, velocity=47, pitch=69)
Item(name=Note, start=3624, end=3686, velocity=50, pitch=73)
Item(name=Note, start=3747, end=3840, velocity=26, pitch=67)
Item(name=Note, start=3747, end=3809, velocity=56, pitch=71)
3840

3840
Item(name=Note, start=3870, end=3901, velocity=32, pitch=69)


In [ ]:
events = utils.item2event(groups)
midi_path = file_name+'_Velocity Changed'+'.midi'
utils.write_midi_events(None, None, midi_path, prompt_path=None, events=events)

sample_data/[Domenico Scarlatti][Sonata in D Major, K. 118]_Velocity Changed.midi




---

\

# Practice 2: Pop_Music_Transformer

## Download Pre-trained Checkpoints
download two pre-trained checkpoints for generating samples.

* REMI-tempo-checkpoint (428 MB)


In [18]:
!pip install gdown
!gdown --id 1gxuTSkF51NP04JZgTE46Pg4KQsbHQKGo
!unzip REMI-tempo-checkpoint.zip

Downloading...
From: https://drive.google.com/uc?id=1gxuTSkF51NP04JZgTE46Pg4KQsbHQKGo
To: /content/drive/My Drive/GCT634_AI613/HW4/REMI-tempo-checkpoint.zip
100% 449M/449M [00:03<00:00, 129MB/s]
Archive:  REMI-tempo-checkpoint.zip
  inflating: REMI-tempo-checkpoint/checkpoint  
  inflating: REMI-tempo-checkpoint/dictionary.pkl  
  inflating: REMI-tempo-checkpoint/model.data-00000-of-00001  
  inflating: REMI-tempo-checkpoint/model.index  
  inflating: REMI-tempo-checkpoint/model.meta  


## Download MIDI dataset
download MIDI files including local tempo changes and estimated chord. (5 MB) and put in data directory.

* data/train: 775 files used for training models
* data/evaluation: 100 files (prompts) used for the continuation experiments

In [ ]:
!gdown --id 1JUDHGrVYGyHtjkfI2vgR1xb2oU8unlI3
!unzip data.zip

Downloading...
From: https://drive.google.com/uc?id=1JUDHGrVYGyHtjkfI2vgR1xb2oU8unlI3
To: /content/drive/My Drive/GCT634_AI613/HW4/data.zip
100% 4.77M/4.77M [00:00<00:00, 74.8MB/s]
Archive:  data.zip
replace data/evaluation/000.midi? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/evaluation/000.midi  
replace data/evaluation/001.midi? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/evaluation/001.midi  
replace data/evaluation/002.midi? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/evaluation/002.midi  
replace data/evaluation/003.midi? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/evaluation/003.midi  
replace data/evaluation/004.midi? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/evaluation/004.midi  
replace data/evaluation/005.midi? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/evaluation/005.midi  
replace data/evaluation/006.midi? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace 

Create the directories

In [ ]:
mkdir './result/'

mkdir: cannot create directory ‘./result/’: File exists


In [ ]:
mkdir './finetune_train_data/'

mkdir: cannot create directory ‘./finetune_train_data/’: File exists


## Import the required modules

In [20]:
from model import PopMusicTransformer
from glob import glob
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'


## Defines generation functions

You can change various parameters (temperature, top-k..)


What is the function of the inputs "temperature" and "topk"?
It is the temperature-controlled stochastic sampling methods are used for generating text from a trained language model. You can find out more details in the reference paper [CTRL: 4.1 Sampling](https://einstein.ai/presentations/ctrl.pdf).

In [21]:
def Generation():
    # declare model
    model = PopMusicTransformer(
        checkpoint='REMI-tempo-checkpoint',
        is_training=False)
    
    # generate from scratch
    model.generate(
        n_target_bar=16,
        temperature=1.2,
        topk=5,
        output_path='./result/from_scratch.midi',
        prompt=None)

    # generate continuation
    model.generate(
        n_target_bar=16,
        temperature=1.2,
        topk=5,
        output_path='./result/test_continuation.midi',
        prompt='./sample_data/sample_data.midi' 

    )
    
    # close model
    model.close()

In [22]:
Generation()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from REMI-tempo-checkpoint/model
./result/from_scratch.midi
./result/test_continuation.midi


## Tensorflow Reset 

The context is maintained in Colab, you need to reset tf before running other codes.

In [23]:
import tensorflow as tf
tf.reset_default_graph()

## Defines Finetune functions

If you want to make further modifications, you can check the 'model.py'.

In [24]:
def Finetune():
    # declare model
    model = PopMusicTransformer(
        checkpoint='REMI-tempo-checkpoint',
        is_training=True)
    # prepared data
    midi_paths = glob('finetune_traindata/Liszt/*.midi') # you need to revise it
    print(midi_paths)
    training_data = model.prepare_data(midi_paths=midi_paths)


    output_checkpoint_folder = 'REMI-finetune' # your decision
    if not os.path.exists(output_checkpoint_folder):
        os.mkdir(output_checkpoint_folder)
    
    # finetune
    model.finetune(
        training_data=training_data,
        output_checkpoint_folder=output_checkpoint_folder,Epoch_n = 10)

    # close
    model.close()

In [25]:
Finetune()

INFO:tensorflow:Restoring parameters from REMI-tempo-checkpoint/model
[]


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


dictionary.pkl is required for generation based on the Finetune output checkpoint.

In [26]:
import shutil

shutil.copyfile("./REMI-tempo-checkpoint/dictionary.pkl", "./REMI-finetune/dictionary.pkl")

'./REMI-finetune/dictionary.pkl'

## Generation based on finetune checkpoints

In [27]:
import tensorflow as tf
tf.reset_default_graph()

In [28]:

def Generation_with_finetune():
    # declare model
    model = PopMusicTransformer(
        checkpoint='REMI-finetune',
        is_training=False)
    
    # generate from scratch
    model.generate(
        n_target_bar=16,
        temperature=1.2,
        topk=5,
        output_path='./result/finetune_from_scratch.midi',
        prompt=None)

    # generate continuation
    # temperature, https://junseong.oopy.io/paper-review/ctrl
    model.generate(
        n_target_bar=16,
        temperature=3.6,
        topk=15,
        output_path='./result/[Domenico Scarlatti][Sonata in D Major, K. 118]_Velocity Changed_Transfored(Liszt).midi',
        prompt='sample_data/[Domenico Scarlatti][Sonata in D Major, K. 118]_Velocity Changed.midi'  #feel free to change
    )
    
    # close model
    model.close()

In [29]:
Generation_with_finetune()

INFO:tensorflow:Restoring parameters from REMI-finetune/model
./result/finetune_from_scratch.midi
./result/[Domenico Scarlatti][Sonata in D Major, K. 118]_Velocity Changed_Transfored(Liszt).midi


# Task 2  : Fine-tune the model with your own MIDI files

1. Find your favorite piano songs
2. Transcribe the piano songs using the pre-trained onset and frames model or your own model (HW3)
3. Using transcription results to fine-tune the Pop Music Transformer pre-trained model

You should submit  

- **MIDI file used for training**
- **Various generation results**
- **Report** about the learning process and subjective evaluatio

FYI, We are not gonna judge the musical quality of the results


## How to synthesize the audio files?

We strongly recommend using DAW and VSTI : [Reaper](https://www.reaper.fm/) (FREE DAW), [Spitfire audio LABS](https://labs.spitfireaudio.com/#type=&search=&new=true) (Free Piano VSTI)  to open/play the generated MIDI files. Or, you can use [FluidSynth](https://github.com/FluidSynth/fluidsynth) with a [SoundFont](https://sites.google.com/site/soundfonts4u/). However, it may not be able to correctly handle the tempo changes (see fluidsynth/issues/141).